<a href="https://colab.research.google.com/github/KathituCodes/Health-Chatbot/blob/main/Health_Assistant_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Heart Health Assistant**
## Features & Functionality

1. **Integration with ML Model:**
  - Uses the [Heart Disease dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data) to train a Random Forest Classifier
  - Provides real-time risk assessment based on user inputs
  - Scales input data appropriately


2. **UI:**
  - Tabbed interface separating chat and risk assessment
  - Organized input fields in columns
  - Visual feedback for risk levels
  - Metric display for risk percentage


3. **Improved Content:**
  - Questions and answers focused on heart health
  - More detailed responses with structured information
  - Integration of insights from our correlation analysis


4. Additional Features:
  - Caching for model training to improve performance
  - Comprehensive risk assessment form
  - Detailed explanations of results
  - Visual indicators for different risk levels

### Important notes:

We'll need to authorize [ngrok](https://dashboard.ngrok.com/signup) when running in Colab. Follow the instructions that appear when running the code.
The public URL will change each time we run the notebook.
If you make changes to the app, we'll need to restart the runtime and run all

## **Installation**
This step involves installing the necessary libraries (streamlit, pyngrok, and streamlit-option-menu), which are crucial for building and deploying the web app.

These commands install the three essential libraries:

1. Streamlit for creating the web interface,
2. Pyngrok for making the local app available online (so others can access it), and
3. Streamlit-option-menu for adding navigation between different sections of the 3.app.


In [1]:
# Installing the required packages
!pip install streamlit           # Streamlit is a framework to create web apps for machine learning models
!pip install pyngrok             # Pyngrok is used to expose the local Streamlit app to the web via a public URL
!pip install streamlit-option-menu # Option menu for creating navigation in the app

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 11.0 MB/s eta 0:00:00


## **App Code**

Here, write the core logic of your Streamlit app. This involves loading data, training a machine learning model, and creating an interactive interface where users can ask questions or assess their heart disease risk.

In [2]:
%%writefile app.py
# Import necessary libraries
import streamlit as st                     # Streamlit to build the app's interface
import pandas as pd                        # Pandas for data handling
import numpy as np                         # Numpy for numerical computations
from sklearn.model_selection import train_test_split  # For splitting data into training and test sets
from sklearn.preprocessing import StandardScaler      # For scaling numerical features
from sklearn.ensemble import RandomForestClassifier   # Random Forest model for classification

# Function to load the heart disease dataset
def load_data():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
    column_names = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
                   "thalach", "exang", "oldpeak", "slope", "ca", "thal", "target"]
    data = pd.read_csv(url, names=column_names)

    # Replace '?' with NaN
    data = data.replace('?', np.nan)

    # Convert all columns to numeric
    for column in data.columns:
        data[column] = pd.to_numeric(data[column])

    # Fill missing values with median for numeric columns
    for column in data.columns:
        data[column] = data[column].fillna(data[column].median())

    return data

# Function to train the Random Forest model
def train_model(data):
    X = data.drop('target', axis=1)       # Dropping the target column (dependent variable)
    y = data['target'].apply(lambda x: 1 if x > 0 else 0)  # Converting the target to binary (1: heart disease, 0: no heart disease)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scaling the feature variables
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Training a Random Forest classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    return model, scaler  # Return the trained model and the scaler for later use

# Main function to define the Streamlit app layout
def main():
    # Set the app title and introductory text
    st.title("Heart Health Assistant")
    st.write("Your interactive guide to heart health and risk assessment")

    # Load and train the model
    @st.cache_resource
    def load_trained_model():
        data = load_data()
        model, scaler = train_model(data)
        return model, scaler, data

    # Load the model, scaler, and data
    model, scaler, data = load_trained_model()

    # Create two tabs: Chat Assistant and Risk Assessment
    tab1, tab2 = st.tabs(["Chat Assistant", "Risk Assessment"])

    with tab1:
        st.subheader("Ask me about heart health!")
        user_query = st.selectbox("Choose your question:", ["What is heart disease?", "What are the symptoms?", "How can I reduce my risk?"])
        if st.button("Get Answer"):
            if user_query == "What is heart disease?":
                st.write("Heart disease refers to various conditions that affect the heart, such as coronary artery disease.")
            elif user_query == "What are the symptoms?":
                st.write("Symptoms include chest pain, shortness of breath, and fatigue.")
            else:
                st.write("To reduce your risk, maintain a healthy diet, exercise regularly, and avoid smoking.")

    with tab2:
        st.subheader("Heart Disease Risk Assessment")
        st.write("Enter your health information for a risk assessment:")

        # Input fields for health details
        age = st.number_input("Age", 20, 100, 50)
        sex = st.selectbox("Sex", ["Male", "Female"])
        cp = st.selectbox("Chest Pain Type", ["Typical Angina", "Atypical Angina", "Non-anginal Pain", "Asymptomatic"])
        trestbps = st.number_input("Resting Blood Pressure (mmHg)", 90, 200, 120)
        chol = st.number_input("Cholesterol (mg/dl)", 100, 600, 200)

        # Assess risk on button click
        if st.button("Assess Risk"):
            input_data = np.array([[age, 1 if sex == "Male" else 0, {"Typical Angina": 0, "Atypical Angina": 1, "Non-anginal Pain": 2, "Asymptomatic": 3}[cp], trestbps, chol]])
            scaled_data = scaler.transform(input_data)  # Scale the input data
            prediction = model.predict(scaled_data)  # Get the model's prediction
            st.write("The model predicts: " + ("Heart Disease" if prediction == 1 else "No Heart Disease"))

if __name__ == "__main__":
    main()

Writing app.py


This script defines the logic of the app:

- The ***Chat Assistant tab*** allows users to ask health-related questions, and the app responds with predefined answers.
- The ***Risk Assessment tab*** accepts user inputs such as age, cholesterol levels, etc., and uses a Random Forest model to predict whether they might be at risk for heart disease.

## **Run the Streamlit App**

Once the app is written, we can run it locally and make it accessible on the web using Pyngrok.

In [3]:
from pyngrok import ngrok

# Install Ngrok authtoken
!ngrok authtoken 2nsOJTOt5ump7Vz8YhWmAVMCo9U_3FV6cMBUrkg8rGm3RyLQn

# Run Streamlit app
!streamlit run app.py &>/dev/null&

# Create an Ngrok tunnel for port 8501 (Streamlit default port)
public_url = ngrok.connect(8501, "http")

# Print the public URL to access the app
print(public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
NgrokTunnel: "https://163f55324a2c.ngrok-free.app" -> "http://localhost:8501"


This command runs the Streamlit app locally (`streamlit run app.py`) and then

uses localtunnel to create a public URL so others can access your app from the web. The `8501` is the default port for Streamlit apps, and the & allows the two commands to run simultaneously.